In [1]:
!pip -q install deepface==0.0.93 retina-face==0.0.14 mtcnn==0.1.1 \
               opencv-python==4.10.0.84 numpy==1.26.4 pandas==2.2.2 \
               faiss-cpu==1.8.0.post1 --no-cache-dir

In [2]:
PATH_TO_SCRIPT = "drive/MyDrive/EE782_A2"

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [9]:
import cv2
import pickle
import sys, os
from pathlib import Path
from datetime import datetime

sys.path.append(PATH_TO_SCRIPT)

from milestone2 import tree, read_bgr, get_embedding, cosine_sim, list_images, build_enrollment_db, build_person_prototypes, identify_image, record_webcam





In [5]:
THRESHOLD = 0.50  # cosine similarity threshold (tune on your data)


BASE = Path("drive/MyDrive/EE782_A2")
ENROLL_DIR = BASE/"enroll"
TEST_DIR   = BASE/"test"
WEBCAM_VIDEOS_DIR =  BASE/"webcam_videos"
FRAMES_DIR = BASE/"frames"

ENROLL_DIR.mkdir(parents=True, exist_ok=True)
TEST_DIR.mkdir(parents=True, exist_ok=True)
WEBCAM_VIDEOS_DIR.mkdir(parents=True, exist_ok=True)
FRAMES_DIR.mkdir(parents=True, exist_ok=True)

print("Enrollment folder :", ENROLL_DIR)
print("Test folder       :", TEST_DIR)
print("Webcam Videos     :", WEBCAM_VIDEOS_DIR)
print("Frames     :", FRAMES_DIR)



EMB_CACHE  = f"{BASE}/face_db.pkl"
DISTANCE   = "cosine"

Enrollment folder : drive/MyDrive/EE782_A2/enroll
Test folder       : drive/MyDrive/EE782_A2/test
Webcam Videos     : drive/MyDrive/EE782_A2/webcam_videos
Frames     : drive/MyDrive/EE782_A2/frames


In [10]:
if Path(EMB_CACHE).exists():
  with open(EMB_CACHE, "rb") as f:
    db_df = pickle.load(f)
else:
  db_df = build_enrollment_db(enroll_dir=ENROLL_DIR, out_path=EMB_CACHE)
print(db_df.head())

     person                                           img_path  \
0    sameer    drive/MyDrive/EE782_A2/enroll/sameer/Sameer.jpg   
1  souparna  drive/MyDrive/EE782_A2/enroll/souparna/Souparn...   

                                           embedding  
0  [0.026221925, 0.029495025, 0.015111535, -0.016...  
1  [-0.027930845, 0.1080185, -0.026532196, -0.099...  


In [11]:
PERSON_PROTOS = build_person_prototypes(db_df)

In [12]:
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
webcam_video_path = WEBCAM_VIDEOS_DIR/f"video_{timestamp}"
record_webcam(out_path=webcam_video_path, seconds=2)

<IPython.core.display.Javascript object>

PosixPath('drive/MyDrive/EE782_A2/webcam_videos/video_2025-10-10 18:37:29')

In [13]:
stride = 5  # save every 5th frame; use 1 to save all

FRAMES_OUT_DIR = FRAMES_DIR/f"frames_{timestamp}"
FRAMES_OUT_DIR.mkdir(parents=True, exist_ok=True)

cap = cv2.VideoCapture(webcam_video_path)
i = 0
saved = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    if i % stride == 0:
        cv2.imwrite(FRAMES_OUT_DIR/f"frame_{i:05d}.jpg", frame)
        saved += 1
    i += 1
cap.release()
print(f"Saved {saved} frames to {FRAMES_OUT_DIR}")

Saved 12 frames to drive/MyDrive/EE782_A2/frames/frames_2025-10-10 18:37:29


In [14]:
for test_img_path in FRAMES_OUT_DIR.iterdir():
  if test_img_path.exists():
    who, score = identify_image(test_img_path, threshold=THRESHOLD, person_protos=PERSON_PROTOS)
    print(f"Result: {who} (cos sim={score:.3f})")
    print("Action:", "WELCOME" if who != "Unknown" else "ALERT: Unrecognized")
else:
    print("Set test_path to an image path under /content/test to try this cell.")

Result: Unknown (cos sim=0.145)
Action: ALERT: Unrecognized
Result: Unknown (cos sim=0.151)
Action: ALERT: Unrecognized
Result: Unknown (cos sim=0.139)
Action: ALERT: Unrecognized
Result: Unknown (cos sim=0.127)
Action: ALERT: Unrecognized
Result: Unknown (cos sim=0.139)
Action: ALERT: Unrecognized
Result: Unknown (cos sim=0.126)
Action: ALERT: Unrecognized
Result: Unknown (cos sim=0.145)
Action: ALERT: Unrecognized
Result: Unknown (cos sim=0.127)
Action: ALERT: Unrecognized
Result: Unknown (cos sim=0.122)
Action: ALERT: Unrecognized
Result: Unknown (cos sim=0.129)
Action: ALERT: Unrecognized
Result: Unknown (cos sim=0.110)
Action: ALERT: Unrecognized
Result: Unknown (cos sim=0.126)
Action: ALERT: Unrecognized
Set test_path to an image path under /content/test to try this cell.
